In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_census_data = pd.read_csv("resources/nst-est2018-alldata.csv")
raw_census_data.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2017,RDOMESTICMIG2018,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018
0,10,0,0,0,United States,308745538,308758105,309326085,311580009,313874218,...,0.000000,0.000000,2.553948,2.746049,2.701727,2.988276,3.328598,3.321549,2.941086,3.001086
1,20,1,0,0,Northeast Region,55317240,55318430,55380645,55600532,55776729,...,-5.651919,-5.222289,0.845134,0.040762,-0.397011,-0.923951,-2.011735,-2.430459,-1.801582,-1.127222
2,20,2,0,0,Midwest Region,66927001,66929743,66974749,67152631,67336937,...,-2.370672,-2.301663,-1.043009,-0.896575,0.042505,-0.715656,-1.357662,-1.226811,-0.519621,-0.431833
3,20,3,0,0,South Region,114555744,114563045,114867066,116039399,117271075,...,2.963135,2.779373,5.379667,5.836112,5.290067,6.206402,7.328494,7.225046,6.252425,6.148925
4,20,4,0,0,West Region,71945553,71946887,72103625,72787447,73489477,...,1.478565,1.350094,2.689358,3.226360,3.343874,4.148127,5.127995,5.372314,4.164981,3.965769


I want to convert the state FIPS code to state name and I want the POPESTIMATE2016 column

In [5]:
FIPS_list = raw_census_data["STATE"].value_counts()
FIPS_list

0     5
28    1
26    1
25    1
24    1
23    1
22    1
21    1
20    1
19    1
18    1
17    1
16    1
15    1
13    1
12    1
11    1
10    1
9     1
8     1
6     1
5     1
4     1
2     1
1     1
27    1
72    1
56    1
30    1
55    1
54    1
53    1
51    1
50    1
49    1
48    1
47    1
46    1
45    1
44    1
42    1
41    1
40    1
39    1
38    1
37    1
36    1
35    1
34    1
33    1
32    1
31    1
29    1
Name: STATE, dtype: int64

In [6]:
print(len(FIPS_list))

53


In [23]:
state_pops_df = raw_census_data[["STATE", "POPESTIMATE2016", "POPESTIMATE2018"]]
state_pops_df.rename(columns = {
    "STATE": "FIPS"
}, inplace=True)
state_pops_df.head()

C:\Users\Stewart\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,FIPS,POPESTIMATE2016,POPESTIMATE2018
0,0,323071342,327167434
1,0,56058789,56111079
2,0,67996917,68308744
3,0,122401186,124753948
4,0,76614450,77993663


### Scrape Wikipedia for a FIPS state code conversion table

In [7]:
wiki_url = "https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code#FIPS_state_codes"

In [8]:
wiki_tables = pd.read_html(wiki_url)
wiki_tables

[                                 0           1             2  \
 0                             Name  Alpha code  Numeric code   
 1                          Alabama          AL            01   
 2                           Alaska          AK            02   
 3                   American Samoa          AS            60   
 4                 American Samoa *         NaN            03   
 5                          Arizona          AZ            04   
 6                         Arkansas          AR            05   
 7                     Baker Island         NaN            81   
 8                       California          CA            06   
 9                     Canal Zone *         NaN            07   
 10                        Colorado          CO            08   
 11                     Connecticut          CT            09   
 12                        Delaware          DE            10   
 13            District of Columbia          DC            11   
 14                      

In [9]:
FIPS_df = wiki_tables[0]
FIPS_df.head()

,0,1,2,3
0,Name,Alpha code,Numeric code,Status
1,Alabama,AL,01,State; counties
2,Alaska,AK,02,State; boroughs
3,American Samoa,AS,60,Outlying area under U.S. sovereignty
4,American Samoa *,NaN,03,(FIPS 5-1 reserved code)


In [11]:
FIPS_df[3].value_counts()

State; counties                                   48
Minor outlying island territory                    9
(FIPS 5-1 reserved code)                           5
Outlying area under U.S. sovereignty               5
Freely Associated State                            3
Federal district[4]                                1
State; parishes                                    1
Minor outlying island territories (aggregated)     1
State; boroughs                                    1
Status                                             1
Name: 3, dtype: int64

In [12]:
only_state_FIPS_df = FIPS_df[FIPS_df[3].str.contains("State")]
only_state_FIPS_df[3].value_counts()

State; counties            48
Freely Associated State     3
State; parishes             1
State; boroughs             1
Name: 3, dtype: int64

In [13]:
only_state_FIPS_df = only_state_FIPS_df[only_state_FIPS_df[3] != "Freely Associated State"]
only_state_FIPS_df[3].value_counts()

State; counties    48
State; parishes     1
State; boroughs     1
Name: 3, dtype: int64

In [14]:
only_state_proper_col_names = only_state_FIPS_df.rename(columns = {
    0: "Name",
    1: "Alpha",
    2: "FIPS",
    3: "Status"
})
only_state_proper_col_names.head()

,Name,Alpha,FIPS,Status
1,Alabama,AL,01,State; counties
2,Alaska,AK,02,State; boroughs
5,Arizona,AZ,04,State; counties
6,Arkansas,AR,05,State; counties
8,California,CA,06,State; counties


In [15]:
print(len(only_state_proper_col_names["Status"]))

50


In [16]:
only_state_proper_col_names.dtypes

Name      object
Alpha     object
FIPS      object
Status    object
dtype: object

In [17]:
FIPS_conversion_df = only_state_proper_col_names[["Alpha", "FIPS"]]
FIPS_conversion_df["FIPS"] = FIPS_conversion_df["FIPS"].astype("int")
FIPS_conversion_df.head()

C:\Users\Stewart\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Alpha,FIPS
1,AL,1
2,AK,2
5,AZ,4
6,AR,5
8,CA,6


In [19]:
name_list = raw_census_data["NAME"].value_counts()
name_list

Puerto Rico             1
Rhode Island            1
Virginia                1
Massachusetts           1
Oregon                  1
Georgia                 1
Midwest Region          1
Kansas                  1
West Region             1
New Hampshire           1
Kentucky                1
Nevada                  1
Texas                   1
Minnesota               1
Michigan                1
Illinois                1
Maryland                1
Montana                 1
Vermont                 1
North Carolina          1
Louisiana               1
Utah                    1
Idaho                   1
Alaska                  1
South Region            1
North Dakota            1
Iowa                    1
California              1
Tennessee               1
Hawaii                  1
Ohio                    1
South Carolina          1
Northeast Region        1
Maine                   1
Florida                 1
United States           1
Mississippi             1
New York                1
West Virgini

In [20]:
print(len(name_list))

57


In [26]:
state_pops_alpha = pd.merge(FIPS_conversion_df, state_pops_df, how = "left",
                            on = "FIPS")
state_pops_alpha

,Alpha,FIPS,POPESTIMATE2016,POPESTIMATE2018
0,AL,1,4864745,4887871
1,AK,2,741504,737438
2,AZ,4,6945452,7171646
3,AR,5,2990410,3013825
4,CA,6,39209127,39557045
5,CO,8,5540921,5695564
6,CT,9,3578674,3572665
7,DE,10,949216,967171
8,FL,12,20629982,21299325
9,GA,13,10304763,10519475


In [25]:
print(len(state_pops_alpha["FIPS"]))

50


In [29]:
state_pops_alpha.dtypes

Alpha              object
FIPS                int32
POPESTIMATE2016     int64
POPESTIMATE2018     int64
dtype: object

In [30]:
state_pops_alpha.to_csv("resources/population_data_2016_2018.csv")